# NOTES: Kaggle - Intro to Machine Learning

Neste notebook está presente algumas anotação sobre o curso de [intro to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning) do [Kaggle](https://www.kaggle.com/) e revisão do conteúdo e conceitos de Machine Learning

In [1]:
import pandas as pd
from IPython.display import display

## Sobre os Dataset

Este dataset foi construido com mais de 80 variáveis que descrevem cada caracterísiticas das casas em Ames e Iowa. A competição e o curso que utiliza este dataset tem como objetivo fazer predições do preço final dessas casa.

E através desses dado que será monstado o modelo, que é nada mais que o aprendizado do compartamento dos dados, que nesse caso é um modelo de predição.

** O dataset está disponível [aqui](https://www.kaggle.com/c/home-data-for-ml-course/data)

In [2]:
file_path = '../datas/kaggle:home_data_for_ml_course.csv'
data = pd.read_csv(file_path)

display(data.head()) # head() traz os primeiros 5 valores to topo do conjunto de dados/dataset

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Intepretação das descrições dos dados

Dentro do dataset há valores que estão faltando (missing values) e isso ocorre por diversos faltores, como: perdade dados, má coleta de dados, dados inexistentes e etc. 

Porem para ter uma boa descrição do dataset podemos usar o pandar pra descriver ele com o:

**dataframe.decribe()**

Que retorna uma série de dados estatísticos como somatótios, distribuições, dispersções, entro outros e tudo isso excluíndo os valores não existentes.

Presentes no **describe()** temos:

- **count**: somatório daquela características
- **mean**: média daquela características
- **std**: desvio padrão daquela características
- **min**: menor valor daquela características
- **25%**: 1º quatil daquela características
- **50%**: 2º quartl daquela características
- **75%**: 3º quartil daquela características
- **max**: maior daquela características

In [3]:
display(data.describe())

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## Seleção dos dados de predição

Seguindo os tutorias e desafios foi determinado como features/características que serão utilizadas para as predições
    * LotArea
    * YearBuilt
    * 1stFlrSF
    * 2ndFlrSF
    * FullBath
    * BedroomAbvGr
    * TotRmsAbvGrd
    
Entretanto a escolha das features influencia completamente no desempenho e na acurracia/precisão do seu modelo.

In [4]:
y = data.SalePrice

feature_columns = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = data[feature_columns]

## Modelo de Predição: Decision Tree

O modelo é a representação matemático do comportamentos dos seus dados e a partir de usa construção dele consegue fazer classificações e predições. Para a construção de um modelo é preciso seguir os seguintes passo:
- **Definição**: Qual o tipo de modelo? Qual algoritmo utilizar? Quais parêmetros?
- **Fit/Ajuste**: É a contrução do modelo matemática a partir do dados tratado anteriormente
- **Predição**: Feitas as predições do conjunto de dados
- **Avaliação**: A partir das predições é definido a quantidade de acertos/acurácia do modelo

Após esses passo, o modelo pode ser mudado suas definições para ser feito um novo ajuste, predições e avaliações para a encontra dos melhores valores, parâmetros e algoritmos para obter um melhores resultados.

O primeiro algoritmo de machine leaning/aprendizado de máquina escolhido pelo tutorial foi o Decision Tree Regressor/Árvore de Decisão Regeessora. confira mais sobre [aqui](../algorithms/Decision_Tree.ipynb)

A decisions tree utilizada foi a do [scikit learn](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor)

In [5]:
from sklearn.tree import DecisionTreeRegressor

# Definição
model = DecisionTreeRegressor(random_state=1)

# Fit/Ajuste
print("Definição do modelo utilizado pro fit/ajuste:\n\n\t",model.fit(X, y),"\n")

# Predição
print("Predição do Modelo:", model.predict(X.head()))
print("Valores dos dados :", y.head().tolist())

print("\n* Os dados mostrados são os primeiros valores do conjunto/dataset\n")

# Avaliação
print("Precisão/Acurácia: ",model.score(X,y),"%")

Definição do modelo utilizado pro fit/ajuste:

	 DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best') 

Predição do Modelo: [208500. 181500. 223500. 140000. 250000.]
Valores dos dados : [208500, 181500, 223500, 140000, 250000]

* Os dados mostrados são os primeiros valores do conjunto/dataset

Precisão/Acurácia:  0.9999346278363472 %


## Treino

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

In [8]:
model = DecisionTreeRegressor(random_state=1)

print("",model.fit(train_X,train_y))

 DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')


In [9]:
val_predictions = model.predict(val_X)

In [10]:
print(val_predictions[:10])
print(val_y[:10])

[186500. 184000. 130000.  92000. 164500. 220000. 335000. 144152. 215000.
 262000.]
258     231500
267     179500
288     122000
649      84500
1233    142000
167     325624
926     285000
831     151000
1237    195000
426     275000
Name: SalePrice, dtype: int64


## Validação do Modelo

In [11]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y,val_predictions)
print(val_mae)

29652.931506849316


## Aprimorando o Modelo

In [12]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [13]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

scores = {}
for leaf in candidate_max_leaf_nodes:
    scores[leaf] = get_mae(leaf, train_X, val_X, train_y, val_y)
    
best_tree_size = min(scores, key=scores.get)
print(best_tree_size)

100


In [14]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size)

final_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=100, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [15]:
val_predictions = final_model.predict(val_X)
val_mae = mean_absolute_error(val_y,val_predictions)

print(val_mae)

16815.938748057826


## Modelo de Predição: Random Forests

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(n_estimators=100,random_state=1)
forest_model.fit(train_X, train_y)
melb_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, melb_preds))

21857.15912981083
